In [1]:
#https://github.com/amazon-science/efficient-longdoc-classification
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed_everything(1376)

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [ ]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

In [ ]:
configs

In [2]:
configs= pk.load(open("trained-models/bart-base-context-enhance-form1/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_max_len_list = [context_max_len]#,300,450]
context_sampling_bounds=(0.1, 0.45)

In [ ]:
configs.model_base

In [3]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
        context_max_len_list= context_max_len_list,#is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 853.14it/s]


In [4]:
generator = model_builder()
train_model_path = "trained-models/bart-base-context-enhance-form1/checkpoint-46761/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.5.context_enforcer._wlc.bias', 'encoder.layers.2.context_enforcer._wrc.bias', 'encoder.layers.5.context_enforcer._wlc.weight', 'encoder.layers.5.context_enforcer._right_mha.in_proj_weight', 'encoder.layers.4.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.2.context_enforcer._wlc.weight', 'encoder.layers.5.context_enforcer._left_mha.out_proj.bias', 'encoder.layers.5.context_enforcer_layer_norm.bias', 'encoder.layers.4.context_enforcer._wrc.weight', 'encoder.layers.4.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.5.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.4.context_enforcer._left_mha.out_proj.weight', 'encoder.layers.4.context_enforcer._right_mha.out_proj.bias', 'encoder.layers.3.context_enforcer._wlc.weight', 'encoder.layers.3.context_enforcer._wlc.bias', 'encoder.layers.4.context_enforcer

In [9]:
te1= test_dataset[693]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [ ]:
example= {'document':"""
           Have you been rocking "The Rachel" for the last twenty years? If so, then you're overdue for a trendier haircut that can make the most of your youthful face and features. 
           Check out some stylish magazines or even some celebrity gossip magazines and see what hairstyles are popular these days. 
           You don't have to go for something ultra-trendy if that's not your thing, but getting a haircut that suits you better than your old one can make you look a decade younger.
           """}
te1=test_dataset._process_data(example)
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [10]:
#seed_everything(1376)
boundary_sample =  (0.15, 0.65)
seq_len = te1.input_ids[:1024].shape[0]

boundary_width = int(0.21*seq_len) 
context_boundary = compute_context_boundary(seq_len,
                                            context_sampling_bounds=boundary_sample,
                                            context_max_len=boundary_width)
context_boundary,seq_len,context_max_len

((214, 327), 542, 310)

In [11]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['US rapper Kanye West has told the BBC that he wants to create an apartment for his former bandmate, Kim Kardashian West.']

In [ ]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

In [ ]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

In [ ]:
te1.input_ids.shape

In [ ]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

In [5]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800

In [6]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
from context_enforcement.data.common import write_to_file
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])
def generate():
    test_data_loader = DataLoader(test_dataset,batch_size=12,
                                sampler= SequentialSampler(test_dataset),
                                collate_fn= SmartCollator(
                pad_token_id=train_dataset.tokenizer.pad_token_id,
                max_len=configs.max_seq_len,
                context_max_len=context_max_len,
                context_sampling_bounds=context_sampling_bounds,
            
            ))

    output_summaries =[]
    for batch in tqdm.tqdm(test_data_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        
        seq_len = b_input_ids.shape[1]

        context_boundary = compute_context_boundary(seq_len,
                                                    context_max_len=110)
        
        boundary_mask =  batch.get("boundary",)
        bb=generator.generate(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                #context_boundary=boundary_mask,
                num_beams=10,
                do_sample=False,
                num_return_sequences=1,
                max_length=140)
        sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
        output_summaries+=sentences
    return output_summaries

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
os.makedirs("outputs/bart-context-enforcer-form1/",exist_ok=True)

In [9]:
context_lens = [100, 200, 300, 400, 500, 650]
outputs = {}
results = {}
for cl in context_lens:
    print(f'Generating for the context length: {cl}')
    generator.model.context_max_len = cl
    generator.model.encoder.context_max_len = cl
    rbase_output = generate()
    outputs[cl] = rbase_output
    
    write_to_file(rbase_output[:len(test_dataset)], 
              f"outputs/bart-crossed-context/best_base_final_{cl}")
    
    scores = metrics.compute(predictions=rbase_output,references=targets)
    print(scores)
    
    results[cl]= scores

Generating for the context length: 100


100%|██████████| 945/945 [35:42<00:00,  2.27s/it] 


Done
{'bleu': 0.09830130047126297, 'precisions': [0.3909875458632629, 0.14068852488704173, 0.07098747757034322, 0.03924120881892942], 'brevity_penalty': 0.8835331176063764, 'length_ratio': 0.8898170642919001, 'translation_length': 232212, 'reference_length': 260966, 'meteor': 0.3109221279397272, 'rouge1': 0.3596219099743052, 'rouge2': 0.14852792488427635, 'rougeL': 0.29140592563192924, 'rougeLsum': 0.2913992950050863}
Generating for the context length: 200


100%|██████████| 945/945 [41:39<00:00,  2.65s/it]


Done
{'bleu': 0.11133628908019347, 'precisions': [0.4184407396233675, 0.160670172097324, 0.08394364283183607, 0.047861167177685675], 'brevity_penalty': 0.8684636839496218, 'length_ratio': 0.8764015235701202, 'translation_length': 228711, 'reference_length': 260966, 'meteor': 0.33337683650713795, 'rouge1': 0.38388150460310677, 'rouge2': 0.16659726704699437, 'rougeL': 0.3127130707167, 'rougeLsum': 0.3125732449092926}
Generating for the context length: 350


100%|██████████| 945/945 [52:46<00:00,  3.35s/it]  


Done
{'bleu': 0.1156133336327475, 'precisions': [0.42818556164129634, 0.16854438936457863, 0.08894653987808288, 0.0510762040763339], 'brevity_penalty': 0.8591805289193619, 'length_ratio': 0.8682242131158848, 'translation_length': 226577, 'reference_length': 260966, 'meteor': 0.34080915081480995, 'rouge1': 0.3918857205870998, 'rouge2': 0.1737718254189405, 'rougeL': 0.31982054386693415, 'rougeLsum': 0.31971051841869136}
Generating for the context length: 720


 31%|███▏      | 297/945 [21:36<44:37,  4.13s/it]  

In [ ]:
base_output = generate()

In [ ]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800
base_output8 = generate()

In [ ]:
generator.model.context_max_len= 600
generator.model.encoder.context_max_len= 600
base_output6 = generate()

In [ ]:
generator.model.context_max_len= 450
generator.model.encoder.context_max_len= 450
base_output45 = generate()

In [ ]:
#os.makedirs("outputs/multi_context/")

In [ ]:
#context_enforcement/data/common.py

write_to_file(base_output8[:len(test_dataset)], 
              "outputs/multi_context/best_base_final_800")

In [ ]:
targets[690],base_output[690]#,base_output_8[690],base_output_45[690]#output_summaries2[690],output_summaries[690],output_summaries6[690],output_summaries8[690]

In [ ]:
scores = metrics.compute(predictions=base_output8,references=targets)
scores

In [ ]:
scores45 = metrics.compute(predictions=base_output45,
                           references=targets)
scores45

In [ ]:
{'bleu': 0.13001522827171397,
 'precisions': [0.43264780948765,
  0.17764752271710285,
  0.09726789720912225,
  0.057229256856921935],
 'brevity_penalty': 0.90401138133862,
 'length_ratio': 0.9083367181931746,
 'translation_length': 237045,
 'reference_length': 260966,
 'meteor': 0.3623311725177827,
 'rouge1': 0.40646161409356885,
 'rouge2': 0.18746257090958113,
 'rougeL': 0.3322105156464427,
 'rougeLsum': 20.3334181119671774}

In [ ]:
scores

In [ ]:
scores4

In [ ]:
scores5

In [5]:
from pathlib import Path

path= "trained-models/bart-base-context-enhance-form1/"

In [8]:
checkpoints=[s for s in Path(path).rglob("*checkpoint-*")]

[PosixPath('trained-models/bart-base-context-enhance-form1/checkpoint-46761'),
 PosixPath('trained-models/bart-base-context-enhance-form1/checkpoint-29757'),
 PosixPath('trained-models/bart-base-context-enhance-form1/checkpoint-51012')]

In [13]:
path.strip().split('/')

['trained-models', 'bart-base-context-enhance-form1', '']

: 